In [1]:
import numpy as np
import optuna
import torch

from image_classification.models.mlp import MLPClassifier
from image_classification.models.resnet import ResNetClassifier
from layers import ResNetBlock
import lightning.pytorch as pl
import lightning.pytorch.loggers as pl_loggers
import lightning.pytorch.callbacks as pl_callbacks

from util import set_seed
from weight_init import init_for_relu
from sklearn.model_selection import ParameterGrid
from datasets import FashionMNIST
from experiment import train
import sklearn.metrics as skm
import torch

C:\Users\George Tzoupis\anaconda3\envs\ailab\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def show_classification_results(test_dl, model, classnames):
    ypred, ytrue = [], []

    model.eval()
    with torch.inference_mode():
        for x, y in test_dl:
            pred = model(x.to(model.device))
            ypred.extend(pred.argmax(-1).cpu().numpy().flatten())
            ytrue.extend(y.cpu().numpy().flatten())

    print(skm.classification_report(ytrue, ypred, target_names=classnames))
    print(skm.confusion_matrix(ytrue, ypred))

# Param Grid using sklearn

In [ ]:
pl.seed_everything(42, True)

train_dl, valid_dl, test_dl = FashionMNIST.get_dataloaders(batch_size=16,
                                                           pin_memory=True,
                                                           num_workers=4,
                                                           persistent_workers=True)
n_classes = len(train_dl.dataset.classes)

# model = ResNetClassifier(
#     n_classes=n_classes,
#     opt='AdamW',
#     lr=1e-1,
#     wd=1e-4, ).apply(init_for_relu)

param_grid = {
    'n_features': [
        (4, 8, 16, 32, 64, 128),
        (8, 16, 32, 64, 128, 256),
        (16, 32, 64, 128, 256, 512),
    ],
    'lr': [1e-3],
    'wd': [1e-5],
}

for params in ParameterGrid(param_grid):
    # model = MLPClassifier(
    #     input_sz=28 * 28,
    #     n_classes=n_classes,
    #     **params,
    # ).apply(init_for_relu)

    model = ResNetClassifier(
        n_classes=n_classes,
        **params).apply(init_for_relu)

    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        save_top_k=1,
        monitor="accuracy/val",
        mode="max",
        filename="best-{epoch:02d}",
        save_last=True,
    )

    experiment_name = f"ResNet-n_layers-{len(params['n_features'])}"
    tb_logger = pl_loggers.TensorBoardLogger(save_dir='./results',
                                             name=experiment_name,
                                             sub_dir=model.__class__.__name__)
    trainer = pl.Trainer(max_epochs=10,
                         # limit_train_batches=10,
                         # limit_val_batches=10,
                         callbacks=[
                             checkpoint_callback,
                         ],
                         logger=tb_logger,
                         )
    trainer.fit(model=model,
                train_dataloaders=train_dl,
                val_dataloaders=valid_dl,
                # ckpt_path='./results/fashion_mnist/version_5/checkpoints/best-epoch=04.ckpt'
                )

    show_classification_results(test_dl, model, test_dl.dataset.classes)


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name       | Type       | Params
------------------------------------------
0 | classifier | Sequential | 309 K 
------------------------------------------
309 K     Trainable params
0         Non-trainable params
309 K     Total params
1.237     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=10` reached.


              precision    recall  f1-score   support

 T-shirt/top       0.72      0.81      0.77      1000
     Trouser       0.99      0.93      0.96      1000
    Pullover       0.58      0.62      0.60      1000
       Dress       0.72      0.89      0.80      1000
        Coat       0.56      0.72      0.63      1000
      Sandal       0.93      0.86      0.89      1000
       Shirt       0.46      0.15      0.23      1000
     Sneaker       0.82      0.93      0.87      1000
         Bag       0.94      0.93      0.94      1000
  Ankle boot       0.92      0.91      0.92      1000

    accuracy                           0.77     10000
   macro avg       0.76      0.77      0.76     10000
weighted avg       0.76      0.77      0.76     10000

[[815   2  14 101  13   2  33   3  17   0]
 [  2 931  14  40   9   1   2   0   1   0]
 [ 19   1 616  12 278   0  69   1   3   1]
 [ 41   4   3 889  23   0  36   0   3   1]
 [  2   1 152  89 719   2  29   1   5   0]
 [  0   0   1   4   0 862 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name       | Type       | Params
------------------------------------------
0 | classifier | Sequential | 1.2 M 
------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.920     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    pl.seed_everything(42, True)

    # Data prep
    train_dl, valid_dl, test_dl = FashionMNIST.get_dataloaders(batch_size=16,
                                                               pin_memory=True,
                                                               num_workers=4,
                                                               persistent_workers=True)

    # Model setup
    n_layers = trial.suggest_int("n_layers", 1, 3)
    n_features = [
        trial.suggest_int("n_features_l{}".format(i), 4, 128, log=True) for i in range(n_layers)
    ]
    params = {
        "n_features": n_features,
        "dropout": trial.suggest_float("dropout", 0.2, 0.5),
        "lr": trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        "wd": trial.suggest_float("wd", 1e-5, 1e-1, log=True),
        "opt": trial.suggest_categorical("opt", ["AdamW", "RMSprop"]),
    }
    n_classes = len(train_dl.dataset.classes)
    model = MLPClassifier(
        input_sz=28 * 28,
        n_classes=n_classes,
        **params,
    ).apply(init_for_relu)

    # Trainer setup
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        save_top_k=1,
        monitor="accuracy/val",
        mode="max",
        filename="best-{epoch:02d}",
        save_last=True,
    )
    tb_logger = pl_loggers.TensorBoardLogger(save_dir='./results',
                                             name='fashion_mnist',
                                             sub_dir=model.__class__.__name__)
    trainer = pl.Trainer(max_epochs=5,
                         limit_train_batches=10,
                         limit_val_batches=10,
                         callbacks=[
                             checkpoint_callback,
                         ],
                         logger=tb_logger,
                         )

    # Train
    trainer.fit(model=model,
                train_dataloaders=train_dl,
                val_dataloaders=valid_dl,
                # ckpt_path='./results/fashion_mnist/version_5/checkpoints/best-epoch=04.ckpt'
                )

    return trainer.callback_metrics["accuracy/val"].item()


# Optuna study setup
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=100, timeout=600, n_jobs=1)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    pl.seed_everything(42, True)

    params = {
        "dropout": trial.suggest_float("dropout", 0.1, 0.5, log=True),
        "lr": trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        "wd": trial.suggest_float("wd", 1e-5, 1e-1, log=True),
        "opt": trial.suggest_categorical("opt", ["SGD", "RAdam"]),
    }

    data_module = FashionMNIST.FashionMNISTDataModule()
    n_classes = data_module.n_classes
    model = ResNetClassifier(
        n_classes=n_classes,
        **params).apply(init_for_relu)

    trainer = train(model,
                    data_module,
                    monitored_metric='accuracy/val',
                    mode='max',
                    max_epochs=10,
                    limit_train_batches=200)

    return trainer.callback_metrics["accuracy/val"].item()


pruner = optuna.pruners.HyperbandPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=100, timeout=60 * 60, n_jobs=1)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
data_module = FashionMNIST.FashionMNISTDataModule()
ckpt_path = "results/FashionMNIST/version_2/checkpoints/last.ckpt"
model = ResNetClassifier.load_from_checkpoint(ckpt_path)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    save_top_k=1,
    monitor="accuracy/val",
    mode="max",
    filename="best-{epoch:02d}",
    save_last=True,
)

tb_logger = pl_loggers.TensorBoardLogger(save_dir='./results',
                                         name=data_module.dataset_name,
                                         sub_dir=model.__class__.__name__)

trainer = pl.Trainer(max_epochs=40,
                     callbacks=[
                         checkpoint_callback,
                     ],
                     logger=tb_logger,
                     )

trainer.fit(model,
            data_module,
            ckpt_path=ckpt_path,
            )